# vllm을 활용한 LLM Inference

- LLM Inference는 다양한 툴이 있지만 가장 대표적이고 많은 기능을 제공하는 것은 vLLM입니다.
- Nvidia이 Triton


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: 나만의 LLM 서버 실행시키기</h2>
    <p>자 이제 vllm을 활용해 여러 모델을 직접 추론 및 서빙해보고 성능을 평가까지 해봅니다.</p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 vllm을 이용해 offline inference와 online inference를 실습해봅니다. stream 모드도 실행해봅니다.</p>
    <p>🐕 vllm engine args를 이해하고 quantization을 적용한 모델을 실행시켜봅니다.</p>
    <p>🦁 다양한 모델이나 프레임워크(SGLang)로 서빙해보고 TTFT, TPOT 등 주요한 Metric을 직접 구해봅니다. </p>
</div>

In [1]:
!huggingface-cli login --token ""

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `SLM_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SLM_TOKEN`


In [5]:
import os
os.environ["HF_TOKEN"] = ""

In [2]:
!uv pip install vllm

Using Python 3.11.11 environment at: /usr
Audited 1 package in 92ms


In [3]:
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download
import os

In [5]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [10]:
snapshot_download(
    "facebook/opt-125m",
    local_dir="./facebook__opt-125m",
    token=os.environ["HF_TOKEN"]
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/250M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/7.10k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/251M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

'/content/facebook__opt-125m'

In [11]:
llm = LLM(model="./facebook__opt-125m")

INFO 03-16 11:38:05 config.py:549] This model supports multiple tasks: {'score', 'generate', 'reward', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 03-16 11:38:05 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./facebook__opt-125m', speculative_config=None, tokenizer='./facebook__opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./facebook__opt-125m, num_scheduler_steps=1, 

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-16 11:38:08 model_runner.py:1115] Loading model weights took 0.2389 GB
INFO 03-16 11:38:09 worker.py:267] Memory profiling takes 0.80 seconds
INFO 03-16 11:38:09 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.60GiB
INFO 03-16 11:38:09 worker.py:267] model weights take 0.24GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 34.80GiB.
INFO 03-16 11:38:09 executor_base.py:111] # cuda blocks: 63355, # CPU blocks: 7281
INFO 03-16 11:38:09 executor_base.py:116] Maximum concurrency for 2048 tokens per request: 494.96x
INFO 03-16 11:38:12 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:29<00:00,  1.19it/s]

INFO 03-16 11:38:42 model_runner.py:1562] Graph capturing finished in 29 secs, took 0.12 GiB
INFO 03-16 11:38:42 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 34.12 seconds


In [12]:
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 41.44it/s, est. speed input: 269.73 toks/s, output: 663.88 toks/s]

Prompt: 'Hello, my name is', Generated text: ' Joel, my dad is my friend and we are in a relationship. I am'
Prompt: 'The president of the United States is', Generated text: ' speaking out against the release of some State Department documents which show the Russians were involved'
Prompt: 'The capital of France is', Generated text: ' known as the “Proud French capital”. What is this city'
Prompt: 'The future of AI is', Generated text: ' literally in danger of being taken by any other company.\nAgreed. '


## Online Infernece
- 여기서는 런타임 세션을 한번 재시작하여 GPU 메모리를 비워주시는 게 좋습니다.
- OpenAI 호환 REST API 서버를 실행시켜봅니다.

In [3]:
import os
from huggingface_hub import snapshot_download

In [6]:
snapshot_download(
    "Qwen/Qwen2.5-1.5B-Instruct",
    local_dir="./Qwen__Qwen2.5-1.5B-Instruct",
    token=os.environ["HF_TOKEN"]
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

'/content/Qwen__Qwen2.5-1.5B-Instruct'

In [12]:
!nohup vllm serve ./Qwen__Qwen2.5-1.5B-Instruct > serve.log 2>&1 &

server.log를 확인해 아래처럼 서버가 띄워진 것을 확인하고 아래 샘플 코드를 실행시켜보세요.

```
Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:26<00:00,  1.30it/s]
INFO 03-16 11:49:13 model_runner.py:1562] Graph capturing finished in 27 secs, took 0.21 GiB
INFO 03-16 11:49:13 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 32.37 seconds
INFO 03-16 11:49:14 api_server.py:958] Starting vLLM API server on http://0.0.0.0:8000
INFO 03-16 11:49:14 launcher.py:23] Available routes are:
INFO 03-16 11:49:14 launcher.py:31] Route: /openapi.json, Methods: HEAD, GET
INFO 03-16 11:49:14 launcher.py:31] Route: /docs, Methods: HEAD, GET
INFO 03-16 11:49:14 launcher.py:31] Route: /docs/oauth2-redirect, Methods: HEAD, GET
INFO 03-16 11:49:14 launcher.py:31] Route: /redoc, Methods: HEAD, GET
INFO 03-16 11:49:14 launcher.py:31] Route: /health, Methods: GET
INFO 03-16 11:49:14 launcher.py:31] Route: /ping, Methods: POST, GET
INFO 03-16 11:49:14 launcher.py:31] Route: /tokenize, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /detokenize, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/models, Methods: GET
INFO 03-16 11:49:14 launcher.py:31] Route: /version, Methods: GET
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/chat/completions, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/completions, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/embeddings, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /pooling, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /score, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/score, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/audio/transcriptions, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /rerank, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v1/rerank, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /v2/rerank, Methods: POST
INFO 03-16 11:49:14 launcher.py:31] Route: /invocations, Methods: POST
INFO:     Started server process [8412]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

In [14]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"./Qwen__Qwen2.5-1.5B-Instruct","object":"model","created":1742126004,"owned_by":"vllm","root":"./Qwen__Qwen2.5-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-02481e116ae7418997176a45c899d01f","object":"model_permission","created":1742126004,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [17]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="./Qwen__Qwen2.5-1.5B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "재밌는 농담 좀 해줘"},
    ]
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-b697c6e3e6b64bdb8e5f1bc7728ab925', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='물체 A가 물체 B를 만 across하다. \n\n물체 A: 안녕하세요, 이 이피스와 이름이 다른 밀가루는 어디에 있습니다? 물체 B: 멀리서 보니까 됐겠네요. 물체 A: 뭐가 됐나요? 물체 B: across!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1742126039, model='./Qwen__Qwen2.5-1.5B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=72, prompt_tokens=29, total_tokens=101, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


In [18]:
chat_response.choices[0].message.content

'물체 A가 물체 B를 만 across하다. \n\n물체 A: 안녕하세요, 이 이피스와 이름이 다른 밀가루는 어디에 있습니다? 물체 B: 멀리서 보니까 됐겠네요. 물체 A: 뭐가 됐나요? 물체 B: across!'

In [22]:
chat_response = client.chat.completions.create(
    model="./Qwen__Qwen2.5-1.5B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "재밌는 농담 좀 해줘"},
    ],
    stream=True
)

print("client: Start streaming chat completions...")
printed_reasoning_content = False
printed_content = False

for chunk in chat_response:
    reasoning_content = None
    content = None
    # Check the content is reasoning_content or content
    if hasattr(chunk.choices[0].delta, "reasoning_content"):
        reasoning_content = chunk.choices[0].delta.reasoning_content
    elif hasattr(chunk.choices[0].delta, "content"):
        content = chunk.choices[0].delta.content

    if reasoning_content is not None:
        if not printed_reasoning_content:
            printed_reasoning_content = True
            print("reasoning_content:", end="", flush=True)
        print(reasoning_content, end="", flush=True)
    elif content is not None:
        if not printed_content:
            printed_content = True
            print("\ncontent:", end="", flush=True)
        # Extract and print the content
        print(content, end="", flush=True)

client: Start streaming chat completions...

content:물론이죠! 여기 밥을 잘 먹을 수 있는 방법을 말씀해주시면 대박놀아 보세요.